<a href="https://colab.research.google.com/github/WaiWasabi/Neural-Networks/blob/master/neural_network_with_matrices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import random
import numpy as np

def sigmoid(z):
  return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
  return sigmoid(z)*(1-sigmoid(z))

def cost_derivative(a, y):
  return (a - y)

class Network(object):
  def __init__(self, sizes):
    self.sizes = sizes
    self.num_layers = len(sizes)
    self.biases = [np.random.randn(x, 1) for x in sizes[1:]]
    self.weights = [np.random.randn(x, y) for x, y in zip(sizes[1:], sizes[:-1])]

  def feedforward(self, a):
    for w, b in zip(self.weights, self.biases):
      a = sigmoid(np.dot(w, a) + b)
    return a

  def backprop(self, train_data, train_label):
    nabla_w = [np.zeros(w.shape) for w in self.weights]
    nabla_b = [np.zeros(b.shape) for b in self.biases]
    activation = train_data
    activations = [train_data]
    zs = []
    for w, b in zip(self.weights, self.biases):
      z = np.dot(w, activation) + b
      zs.append(z)
      activation = sigmoid(z)
      activations.append(activation)
    delta = cost_derivative(activations[-1], train_label) * sigmoid_prime(zs[-1])
    nabla_b[-1] = delta
    nabla_w[-1] = np.dot(delta, activations[-2].transpose())

    for i in range(2, self.num_layers):
      delta = np.dot(self.weights[-i].transpose(), delta) * sigmoid_prime(zs[-i])
      nabla_b[-i] = delta
      nabla_w[-i] = np.dot(delta, activations[-(i+1)].transpose())
    return nabla_w, nabla_b

  def update_mini_batch(self, mini_batch, lr):
    nabla_w = [np.zeros(w.shape) for w in self.weights]
    nabla_b = [np.zeros(b.shape) for b in self.biases]
    for train_data, train_label in mini_batch:
      delta_nabla_w, delta_nabla_b = self.backprop(train_data, train_label)
      nabla_w = [nw + dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
      nabla_b = [nb + dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
    self.weights = [w - (lr/len(mini_batch))*nw for w, nw in zip(self.weights, nabla_w)]
    self.biases = [b - (lr/len(mini_batch))*nb for b, nb in zip(self.biases, nabla_b)]

  def SGD(self, data, mini_batch_size, epochs, learning_rate):
    n = len(data)
    for i in range(epochs):
      random.shuffle(data)
      mini_batches = [data[k:(k+mini_batch_size)] for k in range(0, n, mini_batch_size)]
      for mini_batch in mini_batches:
        self.update_mini_batch(mini_batch, learning_rate)
    